In [25]:
import pandas as pd
import numpy as np

import json
from rdkit import Chem
import pandas as pd
import itertools
import sys
from tqdm import tqdm 
from itertools import chain, combinations
import sys
import multiprocessing as mp
from functools import partial
from collections import Counter
tqdm.pandas()
sys.path.append('../')
from common import *

### input and output

In [22]:
###input
ymdb_output_final = '../../../Data/database/ymdb/ymdb_final_result.xlsx'
mnxreac_smile_atom_mapping_rules_file_path = '../../../Results/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'

# mnxreac_smile_atom_mapping_rules_file_path = '../../../Data/rules/MNXreaction_smiles_atommap_rules_filter_add_no_ec.csv'
target_smiles_path = '../../../Results/lipid/target_smiles_lipid.pickle'
target_model_smiles_path = '../../../Results/lipid/yeast_met_lipid.pickle'
yeast8_metabolites_path = '../../../Data/yeast-GEM-final.csv'
total_met_inchikey0_file = '../../../Results/analysis/total_met_inchikey0.pickle'

# uptake_met_path = '../../../Data/ymdb/ymdb_uptake.csv'
cut_off = 0.3
num = 'all'
num_process = 5
cut_off_retrosynthesis_path = f'../../../Data_retrosynthesis/lipid/top{num}_{cut_off}_remain_re/'
cut_off_path = f'../../../Results/lipid/top{num}_{cut_off}_remain_re/'

YMDB_success_met_smile_before_file = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'YMDB_success_met_smile_top{num}_{cut_off}.pickle'
YMDB_fail_met_smile_before_file = f'../../../Results/lipid/top{num}_{cut_off}_re/' + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'

###output
retrosys_smiles_calculate_similarity_file_path_all = f'../../../Data_retrosynthesis/lipid/retrosys_smiles_calculate_similarity_{cut_off}_remain_re.json'

retrosys_smiles_calculate_similarity_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_top{num}_{cut_off}.json'
retrosys_smiles_calculate_similarity_filter_file_path = cut_off_retrosynthesis_path + f'retrosys_smiles_calculate_similarity_filter_top{num}_{cut_off}.json'
rxndb_path = cut_off_retrosynthesis_path + f'RXNdb_top{num}_{cut_off}/'
failedrxn_path = cut_off_retrosynthesis_path + f'failedrxn_top{num}_{cut_off}.pickle' 
rxndb_all_path = cut_off_retrosynthesis_path + f'RXNdb_all_top{num}_{cut_off}.json'
rxndb_drop_path = cut_off_retrosynthesis_path + f'RXNdb_drop_top{num}_{cut_off}/'
rxndb_met_max_score_file = cut_off_path + f'rxndb_met_max_score_pd_top{num}_{cut_off}.csv'
rxndb_to_model_path  = cut_off_path + f'rxndb_to_model_top{num}_{cut_off}_remain.csv'
YMDB_success_met_smile_file = cut_off_path + f'YMDB_success_met_smile_top{num}_{cut_off}_remain.pickle'
YMDB_fail_met_smile_file = cut_off_path + f'YMDB_fail_met_smile_top{num}_{cut_off}.pickle'

In [18]:
yeast_total_smiles = process_yeast_smiles(target_model_smiles_path)
# yeast_total_smiles = [neutralize_charge(x) for x in yeast_total_smiles]
yeast_total_smiles = [x for x in yeast_total_smiles if Chem.MolFromSmiles(x)]
retrorules = pd.read_csv(mnxreac_smile_atom_mapping_rules_file_path)
rules_product_smiles_lst = process_retrorules(retrorules,s='product_smiles')
rules_product_smarts_lst = process_retrorules(retrorules,s='ProductSMARTs')


total_smiles 19742
total_smiles 19741
total_smiles 19741
total_smiles 16523


29158it [00:02, 10090.75it/s]


11131


29158it [00:03, 7678.41it/s] 

16144


In [19]:
lipid_success = load_pickle(YMDB_success_met_smile_before_file)
lipid_success_inchikey0 = [smiles2inchikey0(x) for x in lipid_success]
yeast_total_smiles = [x for x in yeast_total_smiles if smiles2inchikey0(x) not in lipid_success_inchikey0]
len(yeast_total_smiles)

3935

In [26]:

if not os.path.exists(cut_off_path):
    os.makedirs(cut_off_path)
if not os.path.exists(rxndb_path):
    os.makedirs(rxndb_path)
if not os.path.exists(rxndb_drop_path):
    os.makedirs(rxndb_drop_path)

In [20]:
retrosys_smiles_calculate_similarity = process_rules_with_multiprocessing(rules_product_smiles_lst, yeast_total_smiles,num_processes=num_process,cutoff=cut_off)

retrosys_smiles_calculate_similarity_pd = pd.DataFrame(retrosys_smiles_calculate_similarity)
retrosys_smiles_calculate_similarity_pd.to_json(retrosys_smiles_calculate_similarity_file_path_all)



100%|██████████| 11134/11134 [32:02<00:00,  5.79it/s] 


In [21]:
retrorules = merge_smiles_similarity_rule(retrorules, retrosys_smiles_calculate_similarity_pd)
filter_smiles_muti(retrorules,retrosys_smiles_calculate_similarity_filter_file_path,num_process=num_process)
with open(retrosys_smiles_calculate_similarity_filter_file_path, 'r') as json_file:
    retrorules = json.load(json_file) 
retrorules = pd.DataFrame(retrorules)

100%|██████████| 40/40 [10:39<00:00, 15.99s/it]  


In [22]:
process_retrorules_and_save(rxndb_path,failedrxn_path,retrorules,heterologous_met_smiles=None,num_process=25)


100%|██████████| 29164/29164 [29:16:25<00:00,  3.61s/it]     


In [23]:
#drop metabolites that are not in the ymdb or yeast8
drop_rxndb(rxndb_path, rxndb_drop_path, total_met_inchikey0_file, num_processes=30)
#statistic the number of reactions 
statistic_reaction_num(rxndb_path,rxndb_drop_path)

100%|██████████| 17238/17238 [00:09<00:00, 1758.90it/s]


rxndb: 885993664
rxndb_drop: 1813688


In [24]:
#merge the rxndb
rxndb = merge_rxndb(rxndb_drop_path,rxndb_all_path)
rxndb_df = pd.DataFrame(rxndb).transpose()
print(rxndb_df.shape)
rxndb_df = rxndb_df.reset_index().rename(columns={'index':'NO'})


(1813688, 9)


In [10]:
rxndb_all_smiles = parallel_process_rxn_smiles(rxndb_df, n_splits=100,num_process=num_process)


100%|██████████| 100/100 [00:10<00:00,  9.44it/s]


number of metabolites in RXNDB: 12835


In [29]:
#get all the smiles in the yeast8
yeast8_total_smiles = get_GEM_all_smiles(yeast8_metabolites_path)
success_before_smiles = load_pickle(YMDB_success_met_smile_before_file)
yeast8_total_smiles  = yeast8_total_smiles + success_before_smiles

In [30]:
smile_max_score_pd = calculate_save_smiles_max_score(rxndb_all_smiles, yeast8_total_smiles,rxndb_met_max_score_file,num_processes=num_process)


100%|██████████| 12835/12835 [4:07:05<00:00,  1.16s/it]  


In [31]:
# smiles_max_score_tmp_pd = pd.read_csv(rxndb_met_max_score_uptake_file)

smiles_success,success_rxndbid,smiles_max_score_tmp_pd = process_reactions_in_parallel_reactant(rxndb_df, smile_max_score_pd, num_processes=30, num_iterations=1)
# smiles_max_score_tmp_pd.to_csv(rxndb_met_max_score_file,index=None)





100%|██████████| 100/100 [02:13<00:00,  1.33s/it]


Iteration 1 - Current success count: 10387
Iteration 1 - Current success_rxndbid count: 1408066
final success 10387
final success_rxndbid 1408066


In [32]:
save_success_fail_target_smiles(YMDB_fail_met_smile_before_file,smiles_success,YMDB_success_met_smile_file,YMDB_fail_met_smile_file)


success number: 1350
fail number: 974


In [33]:
save_rxndb_to_model(rxndb_df,success_rxndbid,rxndb_to_model_path)


(1408066, 10)
(1408066, 10)
(1408066, 5)
(1408066, 5)
